# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from time import sleep
import time
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [2]:
############################################################
env = UnityEnvironment(file_name='Reacher.app')
# Environments contain brains which are responsible for deciding the actions of their associated agents
# Here we check for the first brain available, and set it as the default brain 
# that we will be controlling from Python.
brain_name = env.brain_names[0]
brain = env.brains[brain_name]                    # get the default brain

############################################################
env_info = env.reset(train_mode=True)[brain_name] 
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
state_size = brain.vector_observation_space_size
states = env_info.vector_observations

print('Number of agents:', num_agents)
print('Size of each action:', action_size)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)
env_info = env.reset(train_mode=True)[brain_name] 

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Train the Agent

In [3]:
def ddpg(n_episodes=300, max_t=1000):
    """Deep Deterministic Policy Gradient
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    Constants
    ======
        AVE_SCORE_TARGET (int): the average of scores in SCORE_WINDOW, above which we consider the problem solved
        SCORE_WINDOW_SIZE (int): number of scores to average over
    """
    SCORE_WINDOW_SIZE = 100
    AVE_SCORE_TARGET = 13
    solved = False                                         # do we have a solution yet?
    best_score = 0                                         # what's our best average score?
    break_on_first_solution = False                        # bail as soon as we meet criteria?
    scores = []                                            # list containing scores from each episode
    variances = []                                         # I want to see the variance of scores in score wind
    scores_window = deque(maxlen=SCORE_WINDOW_SIZE)        # last 100 scores
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        agent.reset()                                      # reset exploratory noise on agent
        score = 0
        for t in range(max_t):
            action = agent.act(state)                      # get action to take
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        avg_score = np.mean(scores_window)
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, avg_score), end="")
        
        if i_episode % SCORE_WINDOW_SIZE == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}\tScore Variance: {:.2f}'.format(i_episode, avg_score, np.var(scores_window)))
            
        if avg_score >= AVE_SCORE_TARGET and not solved:
            solved = True
            best_score = avg_score
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, avg_score))
            agent.save_policy('checkpoint.pth')
            if break_on_first_solution:
                break
            else:
                print('Continuing to learn')
    
    if avg_score > best_score:
        print('\nFinal Model had Average Score: {:.2f}.  Saving model to: final.pth'.format(avg_score))
        agent.save_policy('final.pth')
    return scores

st = time.time()
scores = ddpg()
print("\n\ntotal train time: ", time.time() - st)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/Users/andy.gooden/.pyenv/versions/3.6.15/envs/p1_navigation/lib/python3.6/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 100	Average Score: 0.70	Score Variance: 0.32
Episode 200	Average Score: 0.86	Score Variance: 0.32
Episode 300	Average Score: 1.14	Score Variance: 0.57

Final Model had Average Score: 1.14.  Saving model to: final.pth


AttributeError: 'Agent' object has no attribute 'save_policy'

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```